In [1]:
%load_ext autoreload                                                                          
%autoreload 2  
%env METAPACK_CACHE /Volumes/Cache/metapack
%env ROWGEN_CACHE /Volumes/Cache/metapack
    
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline

sns.set_context('notebook')
mp.jupyter.init()

# Temp, b/c I don't want to research jupyter to get the value from .zshrc

from rowgenerators import get_cache
from rowgenerators.appurl.util import get_cache_name 
get_cache_name(None), get_cache()

env: METAPACK_CACHE=/Volumes/Cache/metapack
env: ROWGEN_CACHE=/Volumes/Cache/metapack


('metapack', OSFS('/Volumes/Cache/metapack'))

In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()

pkg.set_sys_path()
from pylib import *

pkg

# Select tables from the Credential Engine Registry
`credentialengine.org-registry-1.1.2` Last Update: 2023-10-19T20:45:50

_Tables of data extracted from the Credential Engin JSON-LD file_




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* **[program_titles](notebooks/program_titles.ipynb#df)**. Program titles for programs linked to CIP codes

## References
<ul><li> <strong><a href="http://ds.civicknowledge.org.s3.amazonaws.com/credentialengine.org/ce_registry_1697667117_ddfe24e4dcbf402e2cb741986b6676fc.zip">registry_source</a></strong>. Full download of CE registrry data</li><ul>

In [3]:
s = pkg.reference('registry_source').resolved_url
print(type(s), s)
r = s.get_resource()
r

<class 'rowgenerators.appurl.web.web.WebUrl'> http://ds.civicknowledge.org.s3.amazonaws.com/credentialengine.org/ce_registry_1697667117_ddfe24e4dcbf402e2cb741986b6676fc.zip


<ZipUrl file:///Volumes/Cache/metapack/ds.civicknowledge.org.s3.amazonaws.com/credentialengine.org/ce_registry_1697667117_ddfe24e4dcbf402e2cb741986b6676fc.zip>

In [4]:
r.fspath

PosixPath('/Volumes/Cache/metapack/ds.civicknowledge.org.s3.amazonaws.com/credentialengine.org/ce_registry_1697667117_ddfe24e4dcbf402e2cb741986b6676fc.zip')

In [5]:
import json

def visit_nodes(json_obj, path="", json_path="", results=None):
    if results is None:
        results = []
    if isinstance(json_obj, dict):
        for k, v in json_obj.items():
            new_path = f"{path}.{k}" if path else k
            new_json_path = f"{json_path}['{k}']" if json_path else f"['{k}']"
            if not isinstance(v, (dict, list)):
                results.append((new_path, v, new_json_path))
            visit_nodes(v, new_path, new_json_path, results)
    elif isinstance(json_obj, list):
        for i, item in enumerate(json_obj):
            new_path = f"{path}[{i}]"
            new_json_path = f"{json_path}[{i}]"
            if not isinstance(item, (dict, list)):
                results.append((new_path, item, new_json_path))
            visit_nodes(item, new_path, new_json_path, results)
    return results

def getIPT(k, v, p, d):
    if 'instructionalProgramType' in k and k.endswith('@type'):
        p = p.replace("['@type']",'')
        return eval(f"d{p}")
    else:
        return None

def get_en_str(v):

    if isinstance(v, str):
        return v

    for k in ('en', 'en-US', 'en-AU', 'en-GB' ):
        
        try:
            return v[k]
        except KeyError:
            pass
    
        try:
            return v[k.lower()]
        except KeyError:  
            pass
    
    raise KeyError()
    
    

def extract_fields(d):
    
    try:
        d['decoded_resource']['@graph']
    except KeyError:
        return 
    
    for e in d['decoded_resource']['@graph']:

        try:
            type_ = e['@type']
            desc = get_en_str(e['ceterms:description'])
            name = get_en_str(e['ceterms:name'])
            ctid = e['ceterms:ctid']
        except KeyError:
            continue 
        
        for k, v, p in visit_nodes(e):
            ipt = getIPT(k, v, p, e)
            if ipt:
                try:
                    yield {
                        'ctid': ctid,
                        'name': name,
                        'description': desc,
                        'cred_type': type_.replace('ceterms:',''),
                        'cip': ipt['ceterms:codedNotation']
                    }
                except KeyError:
                    continue

import json
from tqdm.auto import tqdm
from pathlib import Path 
from itertools import islice

recs = []

#for e in tqdm(all_files):   
#    fn = e.get_target().fspath

# The +'_d' nonsense only works after the zip file has been expanded and rename,
# or it's been iterated through once

for fn in tqdm(Path(str(r.fspath)+'_d').glob('**/*.json')):
    t = fn.read_text()
    d = json.loads(t)
    recs.extend(list(extract_fields(d)))

df = pd.DataFrame(recs)
df.sample(20)

0it [00:00, ?it/s]

,ctid,name,description,cred_type,cip
32628,ce-d3caf737-df06-44fb-938b-2d1cdae6e0aa,HVAC/R Technician,"Heating, Ventilation, Air Conditioning and Ref...",Certificate,47.0201
18458,ce-af6dedbf-dad2-409e-9741-63c97cb321bc,M.A. in Clinical Psychology,The Master of Arts in Clinical Psychology prog...,MasterDegree,42.2801
591,ce-b7a56a6e-0feb-4f3d-ba33-1241c6c26dc0,Writing Certificate,A program that focuses on writing for applied ...,Certificate,23.1301
40097,ce-45a65276-4970-4353-be23-a64db5a02d99,Business Administration Apprenticeship Program,Business Administration General Insurance - AA...,LearningOpportunityProfile,52.1701
43652,ce-c3fafdbc-677f-4cdf-917f-3814a5406ff9,Hospitality Management,A program that prepares individuals to serve a...,AssociateDegree,52.0901
23720,ce-ccb35384-e62d-4203-9daa-4d3d6d128b06,Marketing and Sales Certificate,A program that focuses on the general process ...,Certificate,52.1801
3921,ce-d8506ef6-3c2f-4a4a-a5ca-0a4f3e389aad,Physical Therapy Aide,Upon successful completion you will be prepare...,LearningOpportunityProfile,51
8605,ce-95287529-26a1-4685-bfbb-358a396e6ade,Network Administration,A program that prepares individuals to manage ...,AssociateDegree,11.1001
43133,ce-56693cac-47ab-4a69-a257-b4e684ca2b4f,Certificate of Achievement in Fundamentals of ...,This certification prepares students to assume...,Certificate,52.0401
7973,ce-5b60326f-ebc1-443b-ab85-f5c1496fd1cf,Multi-Skilled Robotics Technician,Mott Community College's multi-skilled robotic...,Certificate,15.0405
